In [1]:
#from thllib import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
#from thllib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import networkx as nx


import flylib as flb
#from thllib import flylib as flb
import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
from flylib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import pandas as pd
import networkx as nx

from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

/home/imager/.local/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
flynumbers = list(range(1389,1402))
fly_nums = range(1389,1392)
multi_fly_df = util.construct_multi_fly_df(fly_nums)



Making dataframe for fly number 1389
Making dataframe for fly number 1390
Making dataframe for fly number 1391


In [ ]:
### to extract motifs from closed loop data


In [3]:
multi_fly_df.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg2_right,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right
0,0.000000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
1,0.020000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
2,0.040001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
3,0.060001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
4,0.080002,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0


In [ ]:
multi_fly_df